## 环境的配置

1. 首先安装python3.6以上的版本
2. 安装pipenv 
`sudo pip install --user pipenv //这样将pipenv下载到~/local/bin文件中,然后将该文件添加到.profile文件中即可:vim .profile AND export PATH=$PATH:~/.local/bin`
3. 在工程目录下 `pipenv install`
![pipenv](../photo/pipenvinstall.png)

> To activate this project's virtualenv, run pipenv shell
4. 启动pipenv 安装项目包。 `pipenv install flask`

 - 卸载 `pipenv uninstall flask`
 - 退出 exit
 - 软件包依赖关系 pipenv graph

## 工具安装
1. pycharm 

查看源代码 ctrl+b

2. Xampp -> MySQL
3. Navicat 数据库可视化管理工具

## 环境配置
`pipenv --venv` 查看虚拟环境路径
> /home/tzone/.local/share/virtualenvs/fisher-M_H0keWs

## flask唯一url原则
兼容用户路径多加一个/,使用301重定向。
普通访问
![url1](../photo/url1.png)
加/访问
![url2](../photo/url2.png)
遵循唯一url原则。


## 路由的另一种方法和原理

更新代码自动重启服务器：`app.run(debug=True)`

**chrom浏览器有数据缓存**，如果访问**not found**可以清除浏览器缓存chrome://settings/clearBrowserData?search=%E6%B8%85%E9%99%A4%E6%B5%8F%E8%A7%88

#app.add_url_rule('/hello', view_func=hello) #另一种注册路由的方法

> 路由注册原理：

(查看源码 ctrl+b)
![decorator](../photo/decorator.png)



## app.run相关参数与flask配置文件

> host='0.0.0.0' #所有局域网IP均可访问
> port= #修改端口号

flask配置文件相对自由
创建一个config.py ，然后`app.config.from_object('config')`引入配置文件，`app.run(host=app.config['HOST'], debug=app.config['DEBUG'])`指定参数。

这里app.config是一个dict的字类，config中参数名称必须大写，如果小写会被忽略掉。

## if __name__= '__main__':

![main](../photo/main.png)

```python
if __name__ == '__main__':
    # 生产环境 nginx + uwsgi
    app.run(host=app.config['HOST'], debug=app.config['DEBUG'])

```
一般使用nginx接受web请求，然后使用uwssgi加载相关模块文件启动flask相关代码，所以生产环境下app.run不会执行的。

## 视图函数renturn response对象

视图函数返回值是一个response对象
```python
from flask import make_response

def hello():
    headers = {
        'content-type': 'text/plain',
        'location': 'https://www.baidu.com'
    }
    response = make_response('<html></html>', 301)
    response.headers = headers
    return response
```
如果要提供json格式的数据
在response headers：`content-type: 'application/json'`

**另一种相对简单返回方法**：
`return 'content',301,headers` #没有使用response对象(本质都是resposne)，这里逗号分割的参数，是一个元祖，如果要返回cookie也是调用response下的setcookie方法。


## 路由传参
1. `@app.route('/book/search/<q>/<page>')`

## 深度理解flask路由

> 1. 如何通过url访问到视图函数的？

url - endpoint - viewfunction

一个url会有一个endpoint还会有一个视图函数。

endpoint用来反向构建url。

`app.add_url_rule(‘url’,view_func=,endpoint=) `如果不传endpoint则会把视图函数的函数名作为endpoint的默认值。

### 调试
>  把DEBUG调试模式改为False,启动调试的话，将会把整体代码执行两次，会有一个restart过程。

ctrl B进入源码,在app/web/book.py app.route()源码处打一个断点，


## Request


from flask import request

request:中包含HTTP的请求信息，查询参数，POST参数，remote ip等信息

exampel：
http://**.com?q=1&?page=2

q = request.args['q']

page = request.args['page']

args是一个不可变字典。

通过request.args.to_dict()可以将args转换为一个可变字典。

# wtforms参数验证

> pip install wtforms

**验证层**
```python

from wtforms import Form, StringField, IntegerField
from wtforms.validators import Length, NumberRange, DataRequired


class SearchForm(Form):
    q = StringField(validators=[DataRequired(),Length(min=1, max=20)])
    page = IntegerField(validators=[NumberRange(min=1, max=10)], default=1)
```

# 创建数据表

1. database first 
2. Model First
3. code first 

模型层 MVC

首先安装flask_sqlalchemy 映射数据库表 pipenv install 

code 只关注业务模型,而不是数据库设计, (数据库只是存储数据的,表关系应该由我们的业务来决定)

ORM对象关系映射 : 包含的问题广阔,包含数据的查询删除和更新.

code first 创建数据的问题

# 难点

## flask working outside application context

## flask 上下文 出入栈

```python

from flask import Flask,current_app,request,Request

app = Flask(__name__) # app: <Flask 'test'>

a = current_app   # a:<localProxy unbound>
d = current_app.config['DEBUG']
```

**上下文**

应用上下文 对象  Falsk

请求上下文 对象  Request

Flask 核心对象,注册路由,视图函数, AppContext 对Flask对象的封装(包含对象以外的内容)

Request RequestContext

![flask_app](./flask_app.png)


Request Context入栈时,flask会检测_app_ctx_stack有没有应用上下文对象,如果没有的话,flask会帮你把应用上下文推入到栈中来,所以说 如果是在请求中引用current_app是不需要把app上下文推入到栈中,falsk会帮你推.


```python
app = Flask(__name__)
ctx  = app.app_context()
ctx.push()
a = current_app
d = current_app.config['DEBUG']
ctx.pop()
```

```python

with app.app_context():   # app_context返回一个上下文管理器 AppContext(object) 实现了__enter__和__exit__方法 push()和pop()
    a = current_app
    d = current_app.config['DEBUG']
```

**with 语句**

对实现了上下文协议的对象使用with,

上下文管理器.实现了 `__enter__ 和 __exit__ `

上下文表达式返回上下文管理器

1. 连接数据库  写到 `__enter__`
2. sql ()
3. 释放资源(释放数据库连接)  写到 `__exit__`
try  :sql
except
finally 释放资源

```python

#文件读写
try:
    f = open(r'path')
    print(f.read())
finally:
    f.close()
    
with open(r'') as f:   #open(r'')上下文表达式
    print(f.read())
```

```python
class A:
    def __enter__(self):
        a = 1
        # return a
    def __exit__(self):
        b = 2
with A() as obj_A: #obj_A 是一个空对象 是__enter__方法返回的对象 
    pass
```

----
```python
class MyResource:
    def __enter__(self):
        print('connect to resource')
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_tb:
            print('process exception')
        else:
            print('No exception')
        print('close resource connection')
        # return True
        return False  # no return也是抛出异常

    def query(self):
        print('query data')

try:
    with MyResource() as resource:
        1/0
        resource.query()
except Exception as e:
    print('catch')
    pass
```


# 多线程

python不能充分利用多核cpu优势

GIL全局解释器锁

锁机制:    

        细粒度锁: 程序员  主动加锁
        粗粒度锁: 解释器 GIL bytecode单位执行

GIL 是 cpython解释器的实现 在jpython没有实现

可以开启多进程,使用进程通信技术.(难)

```python
import time
import threading


def worker():
    print('I am thread')
    t = threading.current_thread()
    time.sleep(8)
    print(t.getName())


new_t = threading.Thread(target=worker, name='test_thread')
new_t.start()


t = threading.current_thread()
print(t.getName())  # MainThread
```

**对于IO密集型的程序,多线程是有意义的**

CPU密集型程序: 程序严重依赖CPU计算
IO密集型程序: 查询数据库,请求网络资源,读写文件.

IO密集型的程序的重要特征就是等待.  等待数据返回的时间可以让其他程序执行(多线程)


## flask 多线程

 flask web框架
 
 flask 内置 webserver 默认以**单进程单线程**方式. (方便调试的)
 
 **线程隔离思想** : 不同的请求对应不同的线程,不同的线程在字典中隔离; request = {thread_key1:Request1,...}
 
 
在flask中 werkzeug库 local模块 Local使用字典的方式实现了线程隔离

Local是一个线程隔离对象,对Local对象的操作是相互不影响的.

线程隔离对象的使用:

---

```python
import threading
import time
from werkzeug.local import Local

my_obj = Local() # 线程隔离的对象
my_obj.b = 1


def worker():
    # 新线程
    my_obj.b = 2
    print('in new thread b is : ' + str(my_obj.b))


new_t = threading.Thread(target=worker)
new_t.start()
time.sleep(1)

# 主线程
print("in main thread b is : " + str(my_obj.b))

#in new thread b is : 2
#in main thread b is : 1
```

---

![flask_app](./flask_app.png)

---

Local 可以当做一个普通对象一样,通过点来操作

localStack 线程隔离的栈结构,push方法来操作来生成线性隔离对象.

push,pop,top(不会删除栈顶的元素 属性)

两个线程有两个栈

```python
import threading
import time
from werkzeug.local import Local, LocalStack
my_stack = LocalStack()
my_stack.push(1)
print('in man thread after push, value is : ' + str(my_stack.top))


def worker():
    # 新线程
    print('in new thread before push, value is : ' + str(my_stack.top))
    my_stack.push(2)
    print('in new thread after push, value is : ' + str(my_stack.top))


new_t = threading.Thread(target=worker)
new_t.start()
time.sleep(1)

# 主线程
print('finally, in main thread value is : ' + str(my_stack.top))
```

线程隔离的意义: 使当前线程能够正确引用到他自己所创建的对象,而不是引用到其他线程所创建的对象.

名词:
        
        线程ID号作为key的字典 -> Local -> LocalStack
        AppContext RequestContext 两个上下文 请求进来时候会被推入到 LocalStack 栈中去,请求结束时会被pop出来
        Flask 作为AppContext的属性 Request 同样也作为RequestContext属性封装保存起来
        current_app -> LocalStack.top(栈顶元素).app(的app属性)=Flask
        request -> LocalStack.top.request = Request

 flask